# MATH80629
# Semaine 12 - Prise de décision séquentielle I - Exercices


Auteur: Massimo Caccia massimo.p.caccia@gmail.com <br>

Ces exercices ont été adaptés à partir de : https://github.com/lazyprogrammer/machine_learning_examples/tree/master/rl <br>
ainsi que: https://github.com/omerbsezer/Reinforcement_learning_tutorial_with_demo

## 0. Introduction

Avant de discuter des algorithmes de value iteration et policy iteration, nous allons tester nos connaissances des processus de décision markovien (Markov Decision Process ou MDP). <br>

### 0.1 Tic-Tac-Toe

Prenons l'exemple du&nbsp;: Tic-Tac-Toe (aussi appelé "morpion" et "oxo"). 

Définition: Deux joueurs s'affrontent. Ils doivent remplir chacun à leur tour une case de la grille avec le symbole qui leur est attribué : O ou X. Le gagnant est celui qui arrive à aligner trois symboles identiques, horizontalement, verticalement ou en diagonale. (définition venant de https://fr.wikipedia.org/wiki/Tic-tac-toe)


In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://bjc.edc.org/bjc-r/img/3-lists/TTT1_img/Three%20States%20of%20TTT.png")

**Question:** Si vous vouliez développer un agent pour ce jeu vous devriez d'abord spécifier comment on modélise l'environement. Qu'utiliseriez-vous comme états, actions, fonction de transition et récompenses?

**Réponse:**<br>
L'**espace des états** est une matrice 3x3 ou un vecteur de taille 9 qui indique si une case est: a) vide, b) prise par un X ou c) prise par un O. <br>

Les **actions** sont les 9 cases où vous pouvez jouer. Il y a donc 9 actions possibles en tout. Par contre, toutes les actions ne sont pas toujours disponibles. Notamment, seules les actions ajoutant sur une case vide le sont. <br>

Pour la **fonction de récompense** on pourrait utiliser +1 si vous gagnez, -1 si vous perdez, et 0 pour une partie nulle.

La **fonction de transition** est donnée par la stratégie de l'autre joueur. <br>

### 0.2 Système de recommandation

**Question:** La semaine dernière, nous avons discuté des systèmes de recommandation. Comment faire pour modéliser un système (agent) qui suggère des recommandations en utilisant un MDP?

**Réponse:**

**États:** Vous voudriez encoder dans vos états les préférences de l'utilisateur. Une façon de faire, serait que l'état soit la liste des items déjà consommés par l'utilisateur. 

**Actions:** l'item à recommander (item 1, item 2, ... item n). Le nombre d'actions est donc le nombre d'items dans le catalogue.

**Récompense:** +1 si l'utilisateur consomme l'item et -1 si non. (ce n'est qu'un exemple, on pourrait aussi utiliser une récompense plus complexe.)

**Probabilités de transition:** ça va dépendre de votre utilisateur.

## 1. Value Iteration

Ces exercices testent votre compréhension de l'algorithme Value iteration.

L'algorithme complet se trouve à la [diapo 46](http://www.cs.toronto.edu/~lcharlin/courses/80-629/slides_rl.pdf). <br>

Nous utiliserons l'algorithme pour résoudre un mode grille (Gridworld) similaire à celui présenté à la diapo 12.

### 1.1 Mise en place

In [2]:
#imports

!wget -nc https://raw.githubusercontent.com/lcharlin/80-629/master/week12-MDPs/gridWorldGame.py
    
import numpy as np
from gridWorldGame import standard_grid, negative_grid, print_values, print_policy

File ‘gridWorldGame.py’ already there; not retrieving.



Quelques variables utiles dans notre implémentation. <br>
`SMALL_ENOUGH` pour déterminer quand l'algorithme a convergé<br>
`GAMMA` est le facteur d'actualisation $\gamma$ (discount factor)  dans les diapos (diapo 36) <br>
`ALL_POSSIBLE_ACTIONS` sont les actions disponibles dans l'environement (voir diapo 12): haut (up), bas (down), right (droite), left (gauche).<br>
`NOISE_PROB` défini la stochasticité de l'environement. En d'autres mots, c'est la probabilité qu'une action résulte en un état autre que celui voulu. 

In [3]:
SMALL_ENOUGH = 1e-3 # 
GAMMA = 0.9         # 
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R') # Up, Down, Left, Right
NOISE_PROB = 0.1    # Probabilité que l'agent n'atteigne pas « l'état voulu »

Nous avons écrit une classe pour obtenir un environement en grille (GridWorld). <br>


In [30]:
grid = standard_grid(noise_prob=NOISE_PROB)
print("les récompenses:")
print_values(grid.rewards, grid)

les récompenses:
---------------------------
 0.00| 0.00| 0.00| 1.10|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


Il y a trois états absorbants: (0,3),(1,3) et (1,1)

Nous alors d'abord définir une politique $\pi$ aléatoire. <br>

Rappel: une politique est une fonction qui nous décide de l'action à performer dans chaque état $\pi : S \rightarrow A$.

In [16]:
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

# politique initiale
print("politique initiale:")
print_policy(policy, grid)

politique initiale:
---------------------------
  D  |  R  |  U  | N/A |
---------------------------
  R  | N/A |  D  | N/A |
---------------------------
  D  |  U  |  U  |  L  |


Notez que nous n'avons pas besoin de définir la politique sur les états absorbants.

Nous initialisons ensuite la fonction de valeur de manière aléatoire pour tous les états sauf les états absorbants qui auront une valeur de 0.

In [6]:
np.random.seed(1234) # pour obtenir les mêmes résultats d'une fois à l'autre

V = {}
states = grid.all_states()
for s in states:
    # V[s] = 0
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        # terminal state
        V[s] = 0

# visualiser la fonction de valeur
print_values(V, grid)

---------------------------
 0.44| 0.19| 0.96| 0.00|
---------------------------
 0.80| 0.00| 0.62| 0.00|
---------------------------
 0.78| 0.79| 0.28| 0.27|


### 1.2 L'algorithme de Value iteration - code à compléter

Rappel: Pour Value iteration, l'idée est d'optimiser la fonction de valeur. Une fois obtenue on peut obtenir la meilleure politique pour chaque état. <br>

Nous vous demandons de compléter l'algorithme de Value iteration.<br>

À chaque itération, la valeur $V(s)$ de chaque état $s$ doit être mise à jour avec la formule suivante:

$$
V(s) = \underset{a}{max}\big\{ \sum_{s'}  p(s'|s,a)(r + \gamma*V(s') \big\}
$$


In [31]:
iteration=0
while True:
    print("Itération VI %d: " % iteration)
    print_values(V, grid)
    print("\n\n")
  
    biggest_change = 0
    for s in states:
        old_v = V[s]
       
        # pour chaque état qui n'est pas un état absorbant
        if s in policy:
            new_v = float('-inf')

            # pour chaque action
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                sprime = grid.current_state()
                #  - calculer: [s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
                v = r + GAMMA * V[sprime]
                if v > new_v: # est-ce la meilleure action pour l'instant?
                    new_v = v
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))

    print('\t le plus grand changement de valeur est : %f \n\n' % biggest_change)
    if biggest_change < SMALL_ENOUGH:
        break
    iteration+=1
print_values(V, grid)

Itération VI 0: 
---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.80| 0.89| 0.80|



	 le plus grand changement de valeur est : 0.000000 


---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.80| 0.89| 0.80|


Maintenant que nous avons optimisé la fonction de valeur, nous pouvons obtenir (décoder) la politique optimale:

In [20]:
deterministic_grid = standard_grid(noise_prob=0.)

for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    # on cherche la meilleure action pour chaque état
    for a in ALL_POSSIBLE_ACTIONS:
        deterministic_grid.set_state(s)
        r = deterministic_grid.move(a)
        v = r + GAMMA * V[deterministic_grid.current_state()]
        if v > best_value:
            best_value = v
            best_a = a
    policy[s] = best_a

On peut maintenant visualiser notre politique pour s'assurer qu'elle semble correcte. Notamment, la politique devrait « aller » vers le coin en haut à droite de la grille puisque c'est là qu'il y a la seule récompense positive (+1.1).

In [21]:
print("valeurs:")
print_values(V, grid)
print("\npolitique:")
print_policy(policy, grid)

valeurs:
---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.80| 0.89| 0.80|

politique:
---------------------------
  R  |  R  |  R  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  U  |  R  |  U  |  L  |


## 2. Policy Iteration

Nous vous demandons maintenant de **compléter l'algorithme de policy iteration**. <br>
Vous pouvez trouvez les détails de l'algorithme à la diapo 47. <br>


Nous commençons par définir une politique aléatoire. <br>
Remember that a policy maps states to actions.

In [24]:
policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

# initial policy
print("politique initiale:")
print_policy(policy, grid)

politique initiale:
---------------------------
  R  |  D  |  L  | N/A |
---------------------------
  L  | N/A |  U  | N/A |
---------------------------
  L  |  R  |  L  |  D  |


Et, nous initialisons aussi la fonction de valeur de manière aléatoire

In [25]:
np.random.seed(1234)

# on initialize la fonction de valeur V(s)
V = {}
states = grid.all_states()
for s in states:
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        # état terminal
        V[s] = 0

# visualisation de la fonction de valeur
print_values(V, grid)

---------------------------
 0.44| 0.19| 0.96| 0.00|
---------------------------
 0.80| 0.00| 0.62| 0.00|
---------------------------
 0.78| 0.79| 0.28| 0.27|


### 2.2 Policy iteration - code à compléter

Vous avez maintenant à compléter l'algorithme Policy iteration.<br>
Rappel: l'algorithme fonctionne en deux étapes. <br>

1. on évalue la politique actuelle (*policy evaluation*) en calculant sa fonction de valeur:

$$
V^\pi(s) =  \sum_{s'}  p(s'|s,\pi(s))(r + \gamma*V^\pi(s') 
$$
(Cette partie de l'algorithme vous est donnée.) <br>

2. on améliore la politique (*policy improvement*). Pour chaque état on prend l'action qui donne la meilleure fonction de valeur:

$$
\pi'(s) = \underset{a}{arg max}\big\{ \sum_{s'}  p(s'|s,a)(r + \gamma*V^\pi(s') \big\}
$$

(Vous devez compléter cette partie du code.)<br>

L'algorithme itére ces deux étapes jusqu'à convergence.

In [26]:
iteration=0
# on répète jusqu'à convergence de la politique
while True:
    print("values (iteration %d)" % iteration)
    print_values(V, grid)
    print("policy (iteration %d)" % iteration)
    print_policy(policy, grid)
    print('\n\n')

    # 1. on évalue la politique actuelle (policy evaluation)
    # cette implémentation procède à plusieurs itérations d'évaluation
    # ce n'est pas tout à fait la version de l'algorithme présentée dans les diapos
    # qui n'en fait qu'une.
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]

            # V(s) only has value if it's not a terminal state
            if s in policy:
                a = policy[s]
                grid.set_state(s)
                r = grid.move(a) # reward
                sprime = grid.current_state() # s' 
                V[s] = r + GAMMA * V[sprime]
            biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        if biggest_change < SMALL_ENOUGH:
            break

    #2. on améliore la politique (policy improvement)
    is_policy_converged = True
    for s in states:
        if s in policy:
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            # on trouve la meilleure action 
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                sprime = grid.current_state() 
                v = r + GAMMA * V[sprime]
                if v > best_value:
                    best_value = v
                    new_a = a
            if new_a is None: 
                print('problem')
            policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False

    if is_policy_converged:
        break
    iteration+=1


values (iteration 0)
---------------------------
 0.44| 0.19| 0.96| 0.00|
---------------------------
 0.80| 0.00| 0.62| 0.00|
---------------------------
 0.78| 0.79| 0.28| 0.27|
policy (iteration 0)
---------------------------
  R  |  D  |  L  | N/A |
---------------------------
  L  | N/A |  U  | N/A |
---------------------------
  L  |  R  |  L  |  D  |



values (iteration 1)
---------------------------
 0.00| 0.01| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.01|
policy (iteration 1)
---------------------------
  R  |  U  |  R  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  U  |  L  |  R  |  D  |



values (iteration 2)
---------------------------
 0.00| 0.00| 1.10| 0.00|
---------------------------
 0.00| 0.00| 0.99| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
policy (iteration 2)
---------------------------
  R  |  R  |  U  | N/A |
--------------------

values (iteration 32)
---------------------------
 0.01| 0.01| 1.10| 0.00|
---------------------------
 0.01| 0.00| 0.99| 0.00|
---------------------------
 0.01| 0.01| 0.89| 0.01|
policy (iteration 32)
---------------------------
  R  |  R  |  U  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  R  |  R  |  U  |  L  |



values (iteration 33)
---------------------------
 0.01| 0.01| 0.01| 0.00|
---------------------------
 0.01| 0.00| 0.01| 0.00|
---------------------------
 0.01| 0.01| 0.01| 0.01|
policy (iteration 33)
---------------------------
  R  |  U  |  U  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  R  |  U  |  U  |  D  |



values (iteration 34)
---------------------------
 0.01| 0.01| 0.01| 0.00|
---------------------------
 0.00| 0.00| 0.01| 0.00|
---------------------------
 0.00| 0.00| 0.01|-1.00|
policy (iteration 34)
---------------------------
  R  |  U  |  R  | N/A |
--------------

values (iteration 58)
---------------------------
 0.00| 0.00| 1.10| 0.00|
---------------------------
 0.00| 0.00| 0.99| 0.00|
---------------------------
 0.00| 0.00| 0.89| 0.00|
policy (iteration 58)
---------------------------
  R  |  R  |  R  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  U  |  R  |  D  |  L  |



values (iteration 59)
---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.01| 0.01| 0.01|
policy (iteration 59)
---------------------------
  R  |  R  |  R  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  U  |  L  |  U  |  D  |



values (iteration 60)
---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.65| 0.89| 0.01|
policy (iteration 60)
---------------------------
  R  |  R  |  L  | N/A |
--------------

On peut maintenant visualiser notre politique et vérifier si elle est correcte. Notamment, value iteration et policy iteration devrait donner les mêmes résultats.

In [28]:
print("fonction de valeur finale:")
print_values(V, grid)
print("\npolitique finale:")
print_policy(policy, grid)

fonction de valeur finale:
---------------------------
 0.89| 0.99| 1.10| 0.00|
---------------------------
 0.80| 0.00| 0.99| 0.00|
---------------------------
 0.72| 0.80| 0.89| 0.80|

politique finale:
---------------------------
  R  |  R  |  L  | N/A |
---------------------------
  U  | N/A |  U  | N/A |
---------------------------
  U  |  R  |  U  |  L  |
